# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [67]:
# Load the csv exported in Part I to a DataFrame
# File to load
csvpath = "../Resources/Weather_Output_Final.csv"

# Read CSV and store into Pandas dataframe 
vacation_py_df = pd.read_csv(csvpath)
vacation_py_df.drop(columns= "Unnamed: 0", inplace=True)
vacation_py_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,launceston,-41.45,147.17,39.00,93,75,9.17,AU,1595272653
1,marakkanam,12.20,79.95,83.62,80,98,15.03,IN,1595272654
2,busselton,-33.65,115.33,50.00,66,36,13.85,AU,1595271978
3,quatre cocos,-20.21,57.76,69.80,60,75,10.29,MU,1595272090
4,ushuaia,-54.80,-68.30,32.00,87,0,4.70,AR,1595272186
...,...,...,...,...,...,...,...,...,...
548,amahai,-3.33,128.92,76.96,87,59,5.73,ID,1595272800
549,centralia,46.72,-122.95,82.99,51,1,6.93,US,1595272780
550,fairbanks,64.84,-147.72,66.20,59,1,5.82,US,1595272762
551,umea,63.83,20.26,64.40,88,90,3.36,SE,1595272800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [68]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [72]:
# Store latitude & longitude into locations
locations = vacation_py_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
vacation_py_df = vacation_py_df.dropna()
humidity_percent = vacation_py_df["Humidity"].astype(float)

In [73]:
# Create a heatmap layer to the map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_percent,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [74]:
# Set ideal weather condition: Max temp lower than 80; higher than 70,
ideal_temp = vacation_py_df[(vacation_py_df["Max Temp"] < 80) & (vacation_py_df["Max Temp"] > 70)]

# Wind speed less than 10 mph
ideal_wind_speed = ideal_temp[ideal_temp["Wind Speed"] < 10]

# Zero cloudiness
ideal_cloudiness = ideal_wind_speed[ideal_wind_speed["Cloudiness"] == 0]
ideal_cloudiness

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,paraiba do sul,-22.16,-43.29,72.00,62,0,5.21,BR,1595272666
103,mbanza-ngungu,-5.25,14.87,75.34,37,0,4.72,CD,1595272266
129,asopos,36.73,22.87,75.78,66,0,1.12,GR,1595272686
196,basoko,1.24,23.62,72.84,75,0,3.24,CD,1595272706
214,tiznit,29.58,-9.50,76.62,55,0,6.44,MA,1595272225
230,ugljevik,44.66,19.02,73.00,83,0,2.24,BA,1595272714
240,elbistan,38.21,37.20,76.03,38,0,3.78,TR,1595272717
285,caravelas,-17.71,-39.25,76.46,70,0,8.25,BR,1595272211
302,arak,34.09,49.69,76.64,22,0,5.66,IR,1595272733
314,todos santos,23.45,-110.22,78.01,79,0,8.01,MX,1595272736


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
# Store ideal weather conditions into variable "hotel_df"
hotel_df = ideal_cloudiness.copy()
hotel_df.reset_index(inplace=True, drop=True)
hotel_df["Hotel Name"] = None
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,paraiba do sul,-22.16,-43.29,72.00,62,0,5.21,BR,1595272666,None
1,mbanza-ngungu,-5.25,14.87,75.34,37,0,4.72,CD,1595272266,None
2,asopos,36.73,22.87,75.78,66,0,1.12,GR,1595272686,None
3,basoko,1.24,23.62,72.84,75,0,3.24,CD,1595272706,None
4,tiznit,29.58,-9.50,76.62,55,0,6.44,MA,1595272225,None
5,ugljevik,44.66,19.02,73.00,83,0,2.24,BA,1595272714,None
6,elbistan,38.21,37.20,76.03,38,0,3.78,TR,1595272717,None
7,caravelas,-17.71,-39.25,76.46,70,0,8.25,BR,1595272211,None
8,arak,34.09,49.69,76.64,22,0,5.66,IR,1595272733,None
9,todos santos,23.45,-110.22,78.01,79,0,8.01,MX,1595272736,None


In [79]:
# Set parameters to search for hotels within 5000 meters
target_type = "lodging"
radius = 5000

# Build URL using Google Maps API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for x in range(len(hotel_df.index)):
    city_location = str(hotel_df.iloc[x]["Lat"]) + "," + str(hotel_df.iloc[x]["Lng"])
    parameters = {
        "location": city_location,
        "types": target_type,
        "radius": radius,
        "key": g_key
    }   
    response = requests.get(url, parameters)
    data = response.json()
    try:
        hotel_df.at[x,"Hotel Name"] = data["results"][0]["name"]
        print(data["results"][0]["name"])
    except:
        print("N/A")
# Display
hotel_df.dropna(inplace=True)
hotel_df

Hotel Itaoca
Hotel Full Business
Rachi Boza
Flat Hotel La Benediction
Comsar Energetik
Sword Hotel
Pousada dos Navegantes
Zagros Hotel
Hotel California Todos Santos
Kemping "Malinovoye Ozero"
Prolesok Mini-hotel
Hotel Du Lac
Ilha Flat Hotel
Harmony Inn
Otel KüçükKonak
Hotel Zhana-Tau


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,paraiba do sul,-22.16,-43.29,72.00,62.0,0.0,5.21,BR,1.595273e+09,Hotel Itaoca
1,mbanza-ngungu,-5.25,14.87,75.34,37.0,0.0,4.72,CD,1.595272e+09,Hotel Full Business
2,asopos,36.73,22.87,75.78,66.0,0.0,1.12,GR,1.595273e+09,Rachi Boza
3,basoko,1.24,23.62,72.84,75.0,0.0,3.24,CD,1.595273e+09,Flat Hotel La Benediction
5,ugljevik,44.66,19.02,73.00,83.0,0.0,2.24,BA,1.595273e+09,Sword Hotel
6,elbistan,38.21,37.20,76.03,38.0,0.0,3.78,TR,1.595273e+09,Pousada dos Navegantes
7,caravelas,-17.71,-39.25,76.46,70.0,0.0,8.25,BR,1.595272e+09,Zagros Hotel
8,arak,34.09,49.69,76.64,22.0,0.0,5.66,IR,1.595273e+09,Hotel California Todos Santos
9,todos santos,23.45,-110.22,78.01,79.0,0.0,8.01,MX,1.595273e+09,"Kemping ""Malinovoye Ozero"""
10,malinovoye ozero,51.68,79.78,70.39,66.0,0.0,7.40,RU,1.595273e+09,Prolesok Mini-hotel


In [80]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [81]:
hotel_names = hotel_df["Hotel Name"].tolist()

# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel: {hotel}" for hotel in hotel_names]
)

fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [82]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))